In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

In [2]:
def carrera(url):
    response = requests.get(url, allow_redirects=True)

    # Verifica si la solicitud fue exitosa
    if response.status_code == 200:
        # Analiza el contenido HTML con BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extrae la tabla (ajusta según la estructura de la página)
        table = soup.find('table')

        # Si se encuentra la tabla, la procesa
        if table:
            # Extrae los encabezados
            headers = [header.get_text() for header in table.find_all('th')]
            
            # Extrae las filas
            rows = []
            for row in table.find_all('tr'):
                columns = row.find_all('td')
                row_data = [col.get_text().strip() for col in columns]
                if row_data:
                    rows.append(row_data)
            
            return headers, rows  # Devuelve los encabezados y las filas
        else:
            print('No se encontró una tabla en la página.')
            return None, None  # Devuelve None si no hay tabla
    else:
        print(f'Error al acceder a la página: {response.status_code}')
        return None, None  # Devuelve None si hay error 


In [3]:
# URL de la página principal de las carreras
url_principal = 'https://admision.unmsm.edu.pe/Website20251/A.html'
response = requests.get(url_principal)

# Verifica si la solicitud fue exitosa
if response.status_code == 200:
    # Analiza el contenido HTML con BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Lista para almacenar los enlaces de las carreras
    carrera_links = []

    # Encuentra todos los enlaces (a)
    for a_tag in soup.find_all('a'):
        href = a_tag.get('href')
        # Filtra enlaces que contengan '/A/'
        if href and '/A/' in href:
            carrera_links.append(href)

    # Imprimir la lista de enlaces
    print(carrera_links)
else:
    print(f'Error al acceder a la página: {response.status_code}')

['./A/011/0.html', './A/012/0.html', './A/013/0.html', './A/0141/0.html', './A/0142/0.html', './A/0143/0.html', './A/0144/0.html', './A/015/0.html', './A/022/0.html', './A/023/0.html', './A/031/0.html', './A/0310/0.html', './A/033/0.html', './A/034/0.html', './A/035/0.html', './A/036/0.html', './A/037/0.html', './A/038/0.html', './A/039/0.html', './A/041/0.html', './A/042/0.html', './A/043/0.html', './A/051/0.html', './A/0611/0.html', './A/0612/0.html', './A/0613/0.html', './A/062/0.html', './A/071/0.html', './A/072/0.html', './A/073/0.html', './A/081/0.html', './A/091/0.html', './A/0911/0.html', './A/0912/0.html', './A/092/0.html', './A/0921/0.html', './A/0922/0.html', './A/093/0.html', './A/0931/0.html', './A/0932/0.html', './A/094/0.html', './A/095/0.html', './A/096/0.html', './A/101/0.html', './A/102/0.html', './A/103/0.html', './A/111/0.html', './A/1111/0.html', './A/1112/0.html', './A/112/0.html', './A/1121/0.html', './A/113/0.html', './A/1131/0.html', './A/114/0.html', './A/1141

In [4]:
careers_url = []
for links in carrera_links:
    modified = links.replace('.', 'https://admision.unmsm.edu.pe/Website20251', 1)
    careers_url.append(modified)

print(careers_url)

['https://admision.unmsm.edu.pe/Website20251/A/011/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/012/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/013/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/0141/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/0142/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/0143/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/0144/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/015/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/022/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/023/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/031/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/0310/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/033/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/034/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/035/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/036/0.html', 'https://admision.unmsm.edu.pe/Website20251/A/037/

In [8]:
# Lista para acumular todas las filas y los encabezados
all_rows = []
all_headers = None  # Para almacenar los encabezados (suponiendo que son los mismos en todas las tablas)

for url in careers_url:
    headers, rows = carrera(url)
    if headers and rows:
        # Almacena los encabezados solo una vez
        if all_headers is None:
            all_headers = headers
        all_rows.extend(rows)  # Agrega las filas a la lista total

# Crea un DataFrame con todos los datos
if all_rows:
    df = pd.DataFrame(all_rows, columns=all_headers)

    def corregir_codificacion(texto):
        return texto.replace('Ã\x8d', 'Í') \
                    .replace('Ã\x93', 'Ó') \
                    .replace('Ã\x81', 'Á') \
                    .replace('Ã\x9a', 'Ú') \
                    .replace('Ã\x89', 'É') \
                    .replace('Ã\x89', 'É') \
                    .replace('Ã\x8dA', 'ÍA') \
                    .replace('Ã\x81A', 'Á')

    # Aplicar corrección en las columnas del DataFrame
    df['ESCUELA PROFESIONAL'] = df['ESCUELA PROFESIONAL'].apply(corregir_codificacion)
    # df['ESCUELA SEGUNDA OPCIÓN'] = df['ESCUELA SEGUNDA OPCIÓN'].apply(corregir_codificacion)
    df['APELLIDOS Y NOMBRES'] = df['APELLIDOS Y NOMBRES'].apply(corregir_codificacion)
    
    # Nombre del archivo CSV existente
    file_path = 'resultados_consolidados.csv'
    
    # Verificar si el archivo ya existe
    if os.path.isfile(file_path):
        # Cargar el archivo existente para verificar si tiene datos
        df_existing = pd.read_csv(file_path)
        # Si el archivo ya tiene datos, agregar sin encabezados
        df.to_csv(file_path, mode='a', header=False, index=False)
    else:
        # Si el archivo no existe, crearlo con encabezados
        df.to_csv(file_path, mode='w', header=True, index=False)
else:
    print("No se encontraron datos para exportar.")